In [1]:
# 导入程序所需要的包

# PyTorch需要的包
import torch
import torch.utils.data as DataSet
import torch.nn as nn
import torch.optim
from torch.autograd import Variable
import torch.nn.functional as F
# from torchsummary import summary  # 需要预先下载，在终端输入 pip install torchsummary

# 计算需要的包
import string
import numpy as np
import time

In [2]:
# 读入并展示数据
f = open('../dataset/poems_clean.txt', "r", encoding='utf-8')
poems = []
for line in f.readlines():
    title, poem = line.split(':')
    poem = poem.replace(' ', '')
    poem = poem.replace('\n', '')
    if len(poem) > 0 :
        poems.append(list(poem))
    
print(poems[0][:])

['寒', '随', '穷', '律', '变', '春', '逐', '鸟', '声', '开', '初', '风', '飘', '带', '柳', '晚', '雪', '间', '花', '梅', '碧', '林', '青', '旧', '竹', '绿', '沼', '翠', '新', '苔', '芝', '田', '初', '雁', '去', '绮', '树', '巧', '莺', '来']


In [3]:
# 创建字符编码字典
word2idx = {}
i = 1
for poem in poems:
    for word in poem:
        if word2idx.get(word) == None:
            word2idx[word] = i
            i += 1
            
word2idx

{'寒': 1,
 '随': 2,
 '穷': 3,
 '律': 4,
 '变': 5,
 '春': 6,
 '逐': 7,
 '鸟': 8,
 '声': 9,
 '开': 10,
 '初': 11,
 '风': 12,
 '飘': 13,
 '带': 14,
 '柳': 15,
 '晚': 16,
 '雪': 17,
 '间': 18,
 '花': 19,
 '梅': 20,
 '碧': 21,
 '林': 22,
 '青': 23,
 '旧': 24,
 '竹': 25,
 '绿': 26,
 '沼': 27,
 '翠': 28,
 '新': 29,
 '苔': 30,
 '芝': 31,
 '田': 32,
 '雁': 33,
 '去': 34,
 '绮': 35,
 '树': 36,
 '巧': 37,
 '莺': 38,
 '来': 39,
 '霞': 40,
 '聊': 41,
 '自': 42,
 '怡': 43,
 '晴': 44,
 '弥': 45,
 '可': 46,
 '喜': 47,
 '日': 48,
 '晃': 49,
 '百': 50,
 '色': 51,
 '动': 52,
 '千': 53,
 '池': 54,
 '鱼': 55,
 '跃': 56,
 '不': 57,
 '同': 58,
 '园': 59,
 '还': 60,
 '异': 61,
 '寄': 62,
 '言': 63,
 '博': 64,
 '通': 65,
 '者': 66,
 '知': 67,
 '予': 68,
 '物': 69,
 '外': 70,
 '志': 71,
 '夏': 72,
 '昨': 73,
 '留': 74,
 '灰': 75,
 '秋': 76,
 '箭': 77,
 '今': 78,
 '移': 79,
 '晷': 80,
 '峨': 81,
 '嵋': 82,
 '岫': 83,
 '出': 84,
 '洞': 85,
 '庭': 86,
 '波': 87,
 '渐': 88,
 '起': 89,
 '桂': 90,
 '白': 91,
 '发': 92,
 '幽': 93,
 '岩': 94,
 '菊': 95,
 '黄': 96,
 '灞': 97,
 '涘': 98,
 '运': 99,
 '流': 100,
 '方': 10

In [4]:
len(word2idx)

5545

In [5]:
# 对诗歌进行编码，从原始数据到矩阵
poems_digit = []
for poem in poems:
    poem_digit = []
    for word in poem:
        poem_digit.append(word2idx[word])
    poems_digit.append(poem_digit)
    
print("原始诗歌")
print(poems[3829])
print("\n 编码后的结果")
print(poems_digit[3829][:])

原始诗歌
['春', '眠', '不', '觉', '晓', '处', '处', '闻', '啼', '鸟', '夜', '来', '风', '雨', '声', '花', '落', '知', '多', '少']

 编码后的结果
[6, 2420, 57, 2468, 451, 198, 198, 747, 376, 8, 228, 39, 12, 270, 9, 19, 319, 67, 510, 1941]


In [6]:
# 拆分X、Y变量并处理长短不一问题
# 设置诗歌最大长度为50个字符
maxlen = 50
X = []
Y = []
for poem_digit in poems_digit:
    y=poem_digit[1:]+[0]*(maxlen - len(poem_digit)) # 此处修改y 
    Y.append(y)
    # 将最后一个字符之前的部分作为X，并补齐字符
    x = poem_digit[:-1] + [0]*(maxlen - len(poem_digit))
    X.append(x)
    
print("原始诗歌")
print(poems[3829])
print("变量X")
print(X[3829])
print("变量Y")
print(Y[3829])

原始诗歌
['春', '眠', '不', '觉', '晓', '处', '处', '闻', '啼', '鸟', '夜', '来', '风', '雨', '声', '花', '落', '知', '多', '少']
变量X
[6, 2420, 57, 2468, 451, 198, 198, 747, 376, 8, 228, 39, 12, 270, 9, 19, 319, 67, 510, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
变量Y
[2420, 57, 2468, 451, 198, 198, 747, 376, 8, 228, 39, 12, 270, 9, 19, 319, 67, 510, 1941, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [7]:
# 划分训练集和测试集
# 将所有数据的顺序打乱重排
idx = np.random.permutation(range(len(X)))
X = [X[i] for i in idx]
Y = [Y[i] for i in idx]

# 切分出1/5的数据放入校验集    
validX = X[:len(X) // 5]
trainX = X[len(X) // 5:]
validY = Y[:len(Y) // 5]
trainY = Y[len(Y) // 5:]

In [8]:
'''
将数据转化为dataset，并用dataloader来加载数据。dataloader是PyTorch开发采用的一套管理数据的方法。通常数据储存在dataset中，而对数据的调用则由
dataloader完成。同时，在预处理时，系统已经自动将数据打包成batch，每次调用都取出一批。从dataloader中输出的每一个元素都是一个(x,y)元组，x为输
入张量，y为标签。x和y的第一个维度都是batch_size大小。
'''

# 一批包含64个数据记录。这个数字越大，系统训练时，每一个周期要处理的数据就越多、处理就越快，但总数据量会减少。
batch_size = 64
# 形成训练集
train_ds = DataSet.TensorDataset(torch.IntTensor(np.array(trainX, dtype=int)), torch.IntTensor(np.array(trainY, dtype=int)))
# 形成数据加载器
train_loader = DataSet.DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=4)

# 校验数据
valid_ds = DataSet.TensorDataset(torch.IntTensor(np.array(validX, dtype=int)), torch.IntTensor(np.array(validY, dtype=int)))
valid_loader = DataSet.DataLoader(valid_ds, batch_size=batch_size, shuffle=False, num_workers=4)

In [9]:
'''
实现一个简单的RNN，其构架主要包含3层：输入层，一层隐含层和输出层
'''

class SimpleRNN(nn.Module):
    def __init__(self, output_size, word_num, embedding_size, hidden_size, num_layers=1):
        # 定义
        super(SimpleRNN, self).__init__()
        
        # 一个embedding层
        self.embedding = nn.Embedding(word_num, embedding_size)
        
        # PyTorch的RNN层，batch_first标识可以让输入的张量的第一个维度表示batch指标
        self.rnn = nn.RNN(embedding_size, hidden_size, num_layers, batch_first=True)
        
        # 输出的全连接层
        self.fc = nn.Linear(hidden_size, output_size)
                
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.output_size = output_size
        
    def forward(self, x, hidden):
        # 运算过程
        # 先进行embedding层的计算
        x = self.embedding(x)
        # 从输入到隐含层的计算
        # x的尺寸为：batch_size，num_step，hidden_size
        output, hidden = self.rnn(x, hidden)
        # output的尺寸为：batch_size，maxlen-1, hidden_size
        # 最后一层全连接网络 此处返回每个时间步的数值
        output = self.fc(output)
        output = output.view(-1,output.shape[-1])#为便于后续处理，此处进行展平
        # output的尺寸为：batch_size*(maxlen-1)，output_size
        return output, hidden
    
    def initHidden(self, batch_size):
        # 对隐含单元初始化
        # 尺寸是layer_size，batch_size，hidden_size
        return Variable(torch.zeros(self.num_layers, batch_size, self.hidden_size))

In [10]:
# 获取文本数据集中包含的字符数量
vocab_size = len(word2idx.keys()) + 1

# 给定超参数
lr = 1e-3
epochs = 50

# 生成一个简单的RNN，输入size为49（50-1），输出size为vocab_size（字符总数）
rnn = SimpleRNN(output_size=vocab_size, word_num=vocab_size, embedding_size=64, hidden_size=128)
rnn = rnn.cuda()
criterion = torch.nn.CrossEntropyLoss() #交叉熵损失函数
optimizer = torch.optim.Adam(rnn.parameters(), lr=lr) #Adam优化算法
# 查看模型具体信息
print(rnn)

SimpleRNN(
  (embedding): Embedding(5546, 64)
  (rnn): RNN(64, 128, batch_first=True)
  (fc): Linear(in_features=128, out_features=5546, bias=True)
)


In [11]:
'''
计算预测错误率的函数，pre是模型给出的一组预测结果（batch_size行、num_classes列的矩阵），label是正确标签
'''

def accuracy(pre, label):
    # 得到每一行（每一个样本）输出值最大元素的下标
    pre = torch.max(pre.data, 1)[1]
    # 将下标与label比较，计算正确的数量
    rights = pre.eq(label.data).sum()
    # 计算正确预测所占百分比
    acc = rights.data/len(label)
    return acc.float()

In [12]:
# 模型验证
def validate(model, val_loader):
    # 在校验集上运行一遍并计算损失和准确率
    val_loss = 0
    val_acc = 0
    model.eval()
    for batch, data in enumerate(val_loader):
        init_hidden = model.initHidden(len(data[0]))
        init_hidden = init_hidden.cuda()
        x, y = Variable(data[0]), Variable(data[1])
        x, y = x.cuda(), y.cuda()
        outputs, hidden = model(x, init_hidden)
        y = y.long()
        y = y.view(y.shape[0]*y.shape[1]) #此处修改：展平，对应x的维度
        loss = criterion(outputs, y)
        val_loss += loss.data.cpu().numpy()  
        val_acc += accuracy(outputs, y)
    val_loss /= len(val_loader)  # 计算平均损失
    val_acc /= len(val_loader)  # 计算平均准确率
    return val_loss, val_acc

In [13]:
# 打印训练结果
def print_log(epoch, train_time, train_loss, train_acc, val_loss, val_acc, epochs=10):
    print(f"Epoch [{epoch}/{epochs}], time: {train_time:.2f}s, loss: {train_loss:.4f}, acc: {train_acc:.4f}, val_loss: {val_loss:.4f}, val_acc: {val_acc:.4f}")

In [14]:
# 定义主函数：模型训练
def train(model,optimizer, train_loader, val_loader, epochs=1):
    train_losses = []
    train_accs = []
    val_losses = []
    val_accs = []
    
    for epoch in range(epochs):
        train_loss = 0
        train_acc = 0
        start = time.time()  # 记录本epoch开始时间
        for batch, data in enumerate(train_loader):
            # batch为数字，表示已经进行了几个batch
            # data为一个二元组，存储了一个样本的输入和标签
            model.train() # 标志当前RNN处于训练阶段
            init_hidden = model.initHidden(len(data[0])) # 初始化隐含层单元
            init_hidden = init_hidden.cuda()
            optimizer.zero_grad()
            x, y = Variable(data[0]), Variable(data[1]) # 从数据中提取输入和输出对
            x, y = x.cuda(), y.cuda()
            outputs, hidden = model(x, init_hidden) # 输入RNN，产生输出
            y = y.long()
            y = y.view(y.shape[0]*y.shape[1]) #此处修改：展平，对应x的维度
            loss = criterion(outputs, y) # 带入损失函数并产生loss
            train_loss += loss.data.cpu().numpy()  # 记录loss
            train_acc += accuracy(outputs, y) # 记录acc
            loss.backward() # 反向传播
            optimizer.step() # 梯度更新
        
        end = time.time()  # 记录本epoch结束时间
        train_time = end - start  # 计算本epoch的训练耗时 
        train_loss /= len(train_loader)  # 计算平均损失
        train_acc /= len(train_loader)  # 计算平均准确率             
        val_loss, val_acc = validate(model, val_loader)  # 计算测试集上的损失函数和准确率
        train_losses.append(train_loss)
        train_accs.append(train_acc)
        val_losses.append(val_loss)
        val_accs.append(val_acc) 
        print_log(epoch + 1, train_time, train_loss, train_acc, val_loss, val_acc, epochs=epochs)  # 打印训练结果
    return train_losses, train_accs, val_losses, val_accs

In [15]:
# 模型训练
history = train(rnn, optimizer, train_loader, valid_loader, epochs=epochs)  # 实施训练

Epoch [1/50], time: 4.35s, loss: 4.8651, acc: 0.3455, val_loss: 4.5747, val_acc: 0.3565
Epoch [2/50], time: 4.42s, loss: 4.5105, acc: 0.3597, val_loss: 4.4392, val_acc: 0.3664
Epoch [3/50], time: 4.60s, loss: 4.3686, acc: 0.3683, val_loss: 4.3314, val_acc: 0.3740
Epoch [4/50], time: 4.31s, loss: 4.2485, acc: 0.3755, val_loss: 4.2516, val_acc: 0.3797
Epoch [5/50], time: 4.44s, loss: 4.1522, acc: 0.3808, val_loss: 4.1892, val_acc: 0.3833
Epoch [6/50], time: 4.34s, loss: 4.0741, acc: 0.3847, val_loss: 4.1382, val_acc: 0.3863
Epoch [7/50], time: 4.27s, loss: 4.0024, acc: 0.3881, val_loss: 4.0985, val_acc: 0.3889
Epoch [8/50], time: 4.23s, loss: 3.9449, acc: 0.3911, val_loss: 4.0683, val_acc: 0.3909
Epoch [9/50], time: 4.25s, loss: 3.8973, acc: 0.3936, val_loss: 4.0450, val_acc: 0.3932
Epoch [10/50], time: 4.33s, loss: 3.8560, acc: 0.3960, val_loss: 4.0219, val_acc: 0.3951
Epoch [11/50], time: 4.21s, loss: 3.8162, acc: 0.3985, val_loss: 4.0065, val_acc: 0.3959
Epoch [12/50], time: 4.27s, lo

In [16]:
# 使用RNN写藏头诗
# 初始化藏头诗字符串
poem_incomplete = '深****度****学****习****'
poem_index = [] # 用于记录诗歌创作过程中字符和整数的对应关系
poem_text = '' # 记录诗歌的创作过程，循环结束后应是一首完整的诗

init_hidden = rnn.initHidden(1)
init_hidden = init_hidden.cuda()
for i in range(len(poem_incomplete)):
    # 对poem_incomplete的每个字符做循环
    current_word = poem_incomplete[i]
    if current_word != '*':
        # 若当前的字符不是"*"，使用word2idx字典将其变为一个整数
        index = word2idx[current_word]
    else:
        # 若当前的字符是"*"，需要用RNN模型对其进行预测
        x = poem_index + [0]*(maxlen -1 - len(poem_index)) # 将当前字符与之前的字符拼接形成新的输入序列
        x = torch.IntTensor(np.array([x], dtype=int))
        x = Variable(x)
        x = x.cuda()
        pre, hidden = rnn(x, init_hidden) 
        init_hidden = hidden.data # 更新隐藏状态
        crt_pre = pre[i-1].cpu() # 获取第i个时间步的输出，对应位置i-1
        index = torch.argmax(crt_pre) # 提取最大概率的字符所在的位置，记录其编号
        current_word = [k for k, v in word2idx.items() if v == index][0] # 提取上述编号所对应的字符
    poem_index.append(index) 
    poem_text = poem_text + current_word # 将current_word加到poem_text中

In [17]:
print(poem_text[0:5])
print(poem_text[5:10])
print(poem_text[10:15])
print(poem_text[15:20])

深山河水流
度远山川远
学归来日日
习家贫不得
